In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
from scipy.stats import norm

# binomial_tree_option_priceModel
from scipy.stats import norm
import numpy as np

# Binomial Options Pricing Model for American Call Option
def binomial_tree_option_price(S, K, T, r, sigma, N=10_000):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    prices = np.zeros(N + 1)
    prices[0] = S * d ** N
    for i in range(1, N + 1):
        prices[i] = prices[i - 1] * u / d
    
    option_values = np.maximum(prices - K, 0)
    
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = (p * option_values[j + 1] + (1 - p) * option_values[j]) * np.exp(-r * dt)
            option_values[j] = np.maximum(option_values[j], prices[j] - K)
    
    return option_values[0]

def calculate_greeks(S, K, T, r, sigma, N=100):
    base_price = binomial_tree_option_price(S, K, T, r, sigma, N)
    
    # Delta
    delta_S = 0.01 * S
    price_up = binomial_tree_option_price(S + delta_S, K, T, r, sigma, N)
    price_down = binomial_tree_option_price(S - delta_S, K, T, r, sigma, N)
    delta = (price_up - price_down) / (2 * delta_S)
    
    # Gamma
    price_up_up = binomial_tree_option_price(S + 2 * delta_S, K, T, r, sigma, N)
    gamma = (price_up_up - 2 * base_price + price_down) / (delta_S ** 2)
    
    # Theta
    delta_T = 1/365
    price_T_up = binomial_tree_option_price(S, K, T + delta_T, r, sigma, N)
    theta = (price_T_up - base_price) / delta_T
    
    # Vega
    delta_sigma = 0.01 * sigma
    price_sigma_up = binomial_tree_option_price(S, K, T, r, sigma + delta_sigma, N)
    vega = (price_sigma_up - base_price) / delta_sigma
    
    # Rho
    delta_r = 0.01
    price_r_up = binomial_tree_option_price(S, K, T, r + delta_r, sigma, N)
    rho = (price_r_up - base_price) / delta_r
    
    return delta, gamma, theta, vega, rho

# Retrieve historical data
ticker = "BTCC-B.TO"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()
daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Risk-free rate (annualized)
risk_free_rate = 0.05  # Example rate, adjust as needed

# Current stock price
current_stock_price = data['Close'].iloc[-1]

# Calculate the next 4 Fridays from today
today = datetime.now(pytz.timezone('US/Pacific')).date()
friday_dates = [(today + timedelta(days=(4 - today.weekday()) % 7))]
for i in range(1, 4):
    next_friday = friday_dates[-1] + timedelta(days=7)
    friday_dates.append(next_friday)

# Generate a range of strike prices from -50% to +50% of the current stock price by 5% increments
strike_price_range = np.arange(-0.50, 1.0, 0.05)  # Ends at 0.55 to include 50%
strike_prices = [current_stock_price * (1 + x) for x in strike_price_range]

# Rest of the code for calculating and printing option prices remains the same

for expiry_date in friday_dates:
    days_to_expiration = (expiry_date - today).days / 365  # Convert days to years
    for strike_price in strike_prices:
        call_option_price = binomial_tree_option_price(current_stock_price, strike_price, days_to_expiration, risk_free_rate, annualized_volatility)
        delta, gamma, theta, vega, rho = calculate_greeks(current_stock_price, strike_price, days_to_expiration, risk_free_rate, annualized_volatility)
        print(f"Expiration Date: {expiry_date.strftime('%Y-%m-%d')}, Strike Price: {strike_price:.2f}, Call Option Price: {call_option_price:.2f}, Delta: {delta:.2f}, Gamma: {gamma:.2f}, Theta: {theta:.2f}, Vega: {vega:.2f}, Rho: {rho:.2f}")

[*********************100%%**********************]  1 of 1 completed
C:\Users\falty\AppData\Local\Temp\ipykernel_26268\10643188.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()


Expiration Date: 2024-03-01, Strike Price: 5.26, Call Option Price: 5.27, Delta: 1.00, Gamma: 9.50, Theta: 0.26, Vega: -0.00, Rho: 0.04
Expiration Date: 2024-03-01, Strike Price: 5.79, Call Option Price: 4.74, Delta: 1.00, Gamma: 9.50, Theta: 0.29, Vega: 0.00, Rho: 0.05
Expiration Date: 2024-03-01, Strike Price: 6.32, Call Option Price: 4.21, Delta: 1.00, Gamma: 9.50, Theta: 0.32, Vega: 0.00, Rho: 0.05
Expiration Date: 2024-03-01, Strike Price: 6.84, Call Option Price: 3.69, Delta: 1.00, Gamma: 9.50, Theta: 0.34, Vega: 0.00, Rho: 0.06
Expiration Date: 2024-03-01, Strike Price: 7.37, Call Option Price: 3.16, Delta: 1.00, Gamma: 9.50, Theta: 0.37, Vega: 0.00, Rho: 0.06
Expiration Date: 2024-03-01, Strike Price: 7.90, Call Option Price: 2.64, Delta: 1.00, Gamma: 9.50, Theta: 0.39, Vega: 0.00, Rho: 0.06
Expiration Date: 2024-03-01, Strike Price: 8.42, Call Option Price: 2.11, Delta: 1.00, Gamma: 9.50, Theta: 0.43, Vega: 0.00, Rho: 0.07
Expiration Date: 2024-03-01, Strike Price: 8.95, Call 